<a href="https://colab.research.google.com/github/brunorfo/POC-BIMaster-Brain-Tumor/blob/main/TCC_Brain_tumor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


[link kaggle](https://www.kaggle.com/c/rsna-miccai-brain-tumor-radiogenomic-classification)





In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

import os
workdir_path = '/content/drive/My Drive/TCC'  # Local da pasta onde estão os arquivos de entrada (treino e teste)
os.chdir(workdir_path)
!ls

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
ImageExample.png  test	test.zip  train  train_labels.csv  train.zip


In [ ]:
# Feito a extração das imagens. Necessário apenas a primeira vez que rodar o código.
#!unzip test.zip -d test  # extrai arquivos de teste da pasta compactada.
#!unzip train.zip -d train  # extrai arquivos de treino da pasta compactada.

In [ ]:
import torch
import torchvision
import pandas as pd
import tensorflow as tf
from PIL import Image
import numpy as np

In [ ]:
# Declaração de constantes
TRAIN_FOLDER = 'train'
TEST_FOLDER = 'test'

In [ ]:
print(torch.cuda.is_available())
print(torch.cuda.current_device())
print(torch.cuda.get_device_name(0))

###TODO-Criar bloco com funções acessórias


In [ ]:
# Função para ler todas as imagens de cada paciente
def image_loader(folder:str):
  for dirpath, dirnames, filenames in os.walk(folder):
    for i in range(0, len(dirnames)):
      for patient, empty, photos in os.walk(os.path.join(dirpath, dirnames[i])):
        t = 0
        for files in photos:
          t += 1
          print('faltam', len(dirnames)-i, 'pacientes' )
          print('faltam ', len(photos)-t, 'fotos na pasta', patient )
          file_path = os.path.join(patient, files)
          try:
            yield file_path
            im.close()
          except ValueError as err:
            print(err)
          except TypeError as err:
            print (err)
          except AttributeError as err:
            print(err)

###TODO-Análise exploratória do dataset

In [ ]:
data = pd.read_csv("train_labels.csv", dtype={'BraTS21ID':str, 'MGMT_value':int})

In [ ]:
data.MGMT_value.unique()

array([1, 0])

In [ ]:
data.describe()

,MGMT_value
count,585.000000
mean,0.524786
std,0.499813
min,0.000000
25%,0.000000
50%,1.000000
75%,1.000000
max,1.000000


In [ ]:
data.head()

,BraTS21ID,MGMT_value
0,00000,1
1,00002,1
2,00003,0
3,00005,1
4,00006,1


In [ ]:
with os.scandir('train/00006/FLAIR') as it:
  qt = 0
  for entry in it:
    qt += 1
  print(qt)

68


In [ ]:
im = Image.open('ImageExample.png')
a = np.asarray(im)
print(np.shape(a))
im_tensor = torchvision.transforms.ToTensor()(a)
print(np.shape(im_tensor))

(512, 512)
torch.Size([1, 512, 512])


/usr/local/lib/python3.7/dist-packages/torchvision/transforms/functional.py:114: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:180.)
  img = torch.from_numpy(pic.transpose((2, 0, 1))).contiguous()


In [ ]:
df_images = pd.DataFrame(image_loader(TRAIN_FOLDER), index=None, columns={'patient_id', 'file_name', 'image_size', 'image_mode'})
df_images.to_csv('image_dataframe')


In [ ]:
for dir, file_name, im_size, im_mode in image_loader(TRAIN_FOLDER):
  print(dir,file_name, im_size, im_mode)

###TODO-Ler cada ID de paciente, construir o caminho e ler cada imagem referente ao paciente para servir de input da minha rede neural.



In [ ]:
i = 0
for row in data.itertuples(index=False):
  i += 1
  print('patient ' + row.BraTS21ID)
  print(i)
  if i == 10:
    break